In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install -qU langchain-teddynote


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install langchain_community

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 18.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install langchain.document

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement langchain.document (from versions: none)
ERROR: No matching distribution found for langchain.document

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install jq

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 15.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [7]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력
logging.langsmith("ChatBot_Project")

LangSmith 추적을 시작합니다.
[프로젝트명]
ChatBot_Project


In [8]:
import os

os.environ['LANGCHAIN_PROJECT'] = 'ChatBot_Project'

In [31]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser
from langchain.document_loaders import JSONLoader

# 나머지 코드는 동일하게 유지


########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
search = TavilySearchResults(k=6)

### 1-2. JSON 문서 검색 도구 (Retriever) ###
# JSON 파일 경로
file_path = 'merged_data.json'
# JSONLoader의 jq_schema 설정
jq_schema = """
.[] | {
    "filename": .filename,
    "original": .original,
    "id": .id,
    "date": .date,
    "conference_number": .conference_number,
    "question_number": .question_number,
    "meeting_name": .meeting_name,
    "generation_number": .generation_number,
    "committee_name": .committee_name,
    "meeting_number": .meeting_number,
    "session_number": .session_number,
    "agenda": .agenda,
    "law": .law,
    "qna_type": .qna_type,
    "context": .context,
    "context_learn": .context_learn,
    "context_summary": {
        "summary_q": .context_summary.summary_q,
        "summary_a": .context_summary.summary_a
    },
    "questioner": {
        "name": .questioner_name,
        "ID": .questioner_ID,
        "ISNI": .questioner_ISNI,
        "affiliation": .questioner_affiliation,
        "position": .questioner_position
    },
    "question": {
        "tag": .question.tag,
        "comment": .question.comment,
        "keyword": .question.keyword
    },
    "answerer": {
        "name": .answerer_name,
        "ID": .answerer_ID,
        "ISNI": .answerer_ISNI,
        "affiliation": .answerer_affiliation,
        "position": .answerer_position
    },
    "answer": {
        "tag": .answer.tag,
        "comment": .answer.comment,
        "keyword": .answer.keyword
    }
}
"""

# JSONLoader를 사용하여 파일 로드
loader = JSONLoader(file_path, jq_schema=jq_schema, text_content=False)  # text_content=False 설정
documents = loader.load()

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()

# retriever_tool 정의
retriever_tool = create_retriever_tool(
    retriever,
    name="json_search",  # 도구의 이름을 입력
    description="use this tool to search information from the JSON document",
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
tools = [search, retriever_tool]

########## 2. LLM 을 정의 ##########
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의 ##########
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `json_search` tool for searching information from the JSON document. "
            "If you can't find the information from the JSON document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########
store = {}

def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의 ##########
agent_stream_parser = AgentStreamParser()


In [53]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input":  "2001년 6월 11일에 있었던 회의의 주제는 뭐야?"},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: json_search
query: 2001년 6월 11일 회의 주제
Log: 
Invoking: `json_search` with `{'query': '2001년 6월 11일 회의 주제'}`



[관찰 내용]
Observation: {"filename": "SRC_16\ub300_2001_2001\ub14409\uc6d414\uc77c_\uad6d\uc815\uac10\uc0ac_\uacfc\ud559\uae30\uc220\uc815\ubcf4\ud1b5\uc2e0\uc704\uc6d0\ud68c_0001(029903).xlsx", "original": "http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=029903&fileType=PDF", "id": "029903", "date": "2001\u5e749\u670814\u65e5(\u91d1)", "conference_number": "029903", "question_number": "0001", "meeting_name": "\uad6d\uc815\uac10\uc0ac", "generation_number": "16", "committee_name": "\uacfc\ud559\uae30\uc220\uc815\ubcf4\ud1b5\uc2e0\uc704\uc6d0\ud68c", "meeting_number": "2001", "session_number": "2001\ub14409\uc6d414\uc77c", "agenda": "\uac10\uc0ac\uac1c\uc2dc", "law": "", "qna_type": "\ucd94\ucd9c\ud615", "context": "\uc218\uace0 \ub9ce\uc73c\uc2ed\ub2c8\ub2e4.    9\uc6d4 4\uc77c\uc774\ub2c8\uae4c \uc9c0\uae08\ubd80\ud130 \uc5f4\ud758 \uc804\uc5d0 \uc60